# Topic Modeling

In [1]:
import pandas as pd
import numpy as np
import gensim

C:\Users\Inês Valentim\Anaconda3\lib\site-packages\gensim\utils.py:840: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\Inês Valentim\Anaconda3\lib\site-packages\gensim\utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
### ONLY RUN ONCE ####
# import nltk
# nltk.download('wordnet')

To get good results when performing topic modeling, the documents should be large enough. To avoid emails that are too short, we group the emails by sender (using the SenderPersonId feature) and then consider each document to be the ExtractedSubject + ExtractedBodyText of all the emails belonging to each sender. We also discard emails where both the ExtratedSubject and the ExtractedBodyText are NaN.

Another option would be to group the emails by subject. However, the ExtratedSubject column has a considerable larger number of NaNs than the column SenderPersonId. Thus, we chose to group the emails by sender.

We then apply our text pre-processing pipeline to each document: tokenization, stopwords and digits/numbers removal, lemmatization and removal of tokens that are too small.

In [3]:
from nltk.tokenize import regexp_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# create DataFrame
path_emails = 'hillary-clinton-emails/Emails.csv'
emails = pd.read_csv(path_emails, usecols = ['SenderPersonId', 'ExtractedSubject', 'ExtractedBodyText'])

# drop records where both ExtractedSubject and ExtractedBodyText are NaN
emails.dropna(axis=0, thresh=1, subset=[['ExtractedSubject', 'ExtractedBodyText']], inplace=True)
emails.fillna(' ', inplace = True)

# group emails by sender
emails_raw = []

for sender in emails.groupby('SenderPersonId'):
    grouped_emails = ''
    
    for subject, body in zip(sender[1]['ExtractedSubject'], sender[1]['ExtractedBodyText']):
        grouped_emails += (subject + ' ' + body + ' ')
    
    emails_raw.append(grouped_emails)

# tokenization
email_tokens = []

for email in emails_raw:
    email_tokens.append(regexp_tokenize(email, pattern='\w+'))

# remove stopwords and digits/numbers
stop_words = set(stopwords.words('english'))

# also consider as stopwords those words typically related to emails
stopwords_emails = ['fyi', 'fm', 'am', 'pm', 'n\'t', 'sent', 'from', 'to', 'subject', 'fw', 'fwd', 'fvv',
                    'cc', 'bcc', 'attachments', 're', 'date', 'html', 'php']

stop_words.update(stopwords_emails)

email_clean_tokens = []

for email in email_tokens:
    clean_tokens = [token for token in email if token.lower() not in stop_words and token.isdigit() == False]
    email_clean_tokens.append(clean_tokens)

# lemmatization
wnl = WordNetLemmatizer()

email_lemma = []

for email in email_clean_tokens:
    lemma = [wnl.lemmatize(token.lower()) for token in email]
    email_lemma.append(lemma)
    
# removal of tokens that are too small
email_clean = []

for email in email_lemma:
    clean = [token for token in email if len(token) > 1]
    email_clean.append(clean)

In [7]:
# TODO: explain what we do here

In [4]:
dictionary = gensim.corpora.Dictionary(email_clean)
corpus = [dictionary.doc2bow(email) for email in email_clean]

We perform topic modeling over the corpus by running an LDA model. We run the model for different numbers of topics and we print the 10 most significant words of each topic.

In [5]:
number_topics = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]

for n in number_topics:
    lda_model = gensim.models.ldamodel.LdaModel(corpus, num_topics=n, id2word=dictionary, passes=10)
   
    print('#### Model with', n, 'topics ####')
   
    topic_id = 1
   
    for topic in lda_model.show_topics(num_topics=n, num_words=10, log=False, formatted=False):
        
        string_words = ''
        
        for word in topic[1]:
            string_words += (word[0] + ' ')
        
        print('topic #', topic_id, ':', string_words)

        topic_id += 1

#### Model with 5 topics ####
topic # 1 : state cheryl haiti mill clinton call cdm gov would secretary 
topic # 2 : state would secretary department valmoro time lona assistant need american 
topic # 3 : state call secretary office meeting time department gov talk house 
topic # 4 : woman state would said new year one work also people 
topic # 5 : obama would american president one party said state new time 
#### Model with 10 topics ####
topic # 1 : state department secretary official diplomacy conflict force office government effort 
topic # 2 : state cheryl mill haiti gov cdm call would clinton secretary 
topic # 3 : woman state would also one could work un new people 
topic # 4 : clinton state said one secretary obama president say time policy 
topic # 5 : medium public haiti new work jm state pakistan also communication 
topic # 6 : office secretary state room meeting en arrive route depart department 
topic # 7 : obama would american party president one said republican time new 


In [8]:
# TODO: discussion (best number of topics) 